In [113]:
# Generated synthetic univariate or multivariate time series data
import pandas as pd
import numpy as np

def generate_time_series_data(start_time: str = "2021-01-01 00:00:00", 
                              freq: str = "H", 
                              periods: int = 40, 
                              n_variables: int = 10, 
                              univariate: bool = False) -> pd.DataFrame:
    """
    Generate a univariate or multivariate time series DataFrame.
    
    Args:
        start_time (str): Start datetime.
        freq (str): Pandas frequency string (e.g., 'H' for hourly).
        periods (int): Number of time steps.
        n_variables (int): Number of variables (ignored if univariate=True).
        univariate (bool): Whether to generate univariate (single-column) data.

    Returns:
        pd.DataFrame: Time series data with datetime index.
    """
    index = pd.date_range(start=start_time, periods=periods, freq=freq)

    if univariate:
        data = np.random.randn(periods)
        df = pd.DataFrame(data, index=index, columns=["A"])
    else:
        columns = list("ABCDEFGHIJ")[:n_variables]
        data = np.random.randn(periods, n_variables)
        df = pd.DataFrame(data, index=index, columns=columns)
    
    return df

df_multivariate = generate_time_series_data()
print(df_multivariate)
df_univariate = generate_time_series_data(univariate=True)
print(df_univariate)

                            A         B         C         D         E  \
2021-01-01 00:00:00 -1.409461 -1.467391 -1.237132 -1.188972  0.151228   
2021-01-01 01:00:00 -0.030559 -1.580026  0.100755 -1.127709  0.960680   
2021-01-01 02:00:00  0.738897 -0.283996 -0.436090 -0.358438 -0.194627   
2021-01-01 03:00:00  0.051758 -0.874505 -1.076345 -1.762191 -0.247766   
2021-01-01 04:00:00 -1.649765 -1.180706  0.191715 -1.123570  0.262608   
2021-01-01 05:00:00  0.746877 -0.104188 -0.853755 -1.401118  2.192753   
2021-01-01 06:00:00  0.713362  0.464608 -1.007737 -1.925211 -0.285660   
2021-01-01 07:00:00 -0.905679  0.758190 -0.277418 -0.882158  1.538483   
2021-01-01 08:00:00 -0.267934  0.703125 -0.042448  0.771050  0.011863   
2021-01-01 09:00:00  0.068510 -1.122018 -1.444404 -1.674476 -0.566286   
2021-01-01 10:00:00  1.493815  0.760580 -2.560808 -0.326881 -1.499478   
2021-01-01 11:00:00 -1.786797  0.794970  0.819318 -0.017989 -1.174751   
2021-01-01 12:00:00  0.114068  0.620595  0.889563 -

In [114]:
import torch
import matplotlib.pyplot as plt
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from huggingface_hub import hf_hub_download

from uni2ts.eval_util.plot import plot_single
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
from uni2ts.model.moirai_moe import MoiraiMoEForecast, MoiraiMoEModule

# Model configuration
MODEL = "moirai"  # model name: choose from {'moirai', 'moirai-moe'}
SIZE = "small"  # model size: choose from {'small', 'base', 'large'}
PDT = 4  # prediction length: any positive integer
CTX = 10  # context length: any positive integer
PSZ = 8  # patch size: choose from {"auto", 8, 16, 32, 64, 128}
BSZ = 8  # batch size: any positive integer
TEST = 8  # test set length: any positive integer

gluon_multi = PandasDataset(dict(df_multivariate))
gluon_uni = PandasDataset(dict(df_univariate))
print(gluon_multi)
print(gluon_uni)

PandasDataset<size=10, freq=H, num_feat_dynamic_real=0, num_past_feat_dynamic_real=0, num_feat_static_real=0, num_feat_static_cat=0, static_cardinalities=[]>
PandasDataset<size=1, freq=H, num_feat_dynamic_real=0, num_past_feat_dynamic_real=0, num_feat_static_real=0, num_feat_static_cat=0, static_cardinalities=[]>


In [115]:
# Obtain train and test data
train_multi, test_template_multi = split(
    gluon_multi, offset=-TEST
)  # assign last TEST time steps as test set


train_uni, test_template_uni = split(
    gluon_uni, offset=-TEST
)  # assign last TEST time steps as test set


# Construct rolling window evaluation
test_data_multi = test_template_multi.generate_instances(
    prediction_length=PDT,  # number of time steps for each prediction
    windows=TEST // PDT,  # number of windows in rolling window evaluation
    distance=PDT,  # number of time steps between each window - distance=PDT for non-overlapping windows
)

# Construct rolling window evaluation
test_data_uni = test_template_uni.generate_instances(
    prediction_length=PDT,  # number of time steps for each prediction
    windows=TEST // PDT,  # number of windows in rolling window evaluation
    distance=PDT,  # number of time steps between each window - distance=PDT for non-overlapping windows
)



In [116]:
def moirai_maker(MODEL, gluon_data):
    if MODEL == "moirai":
        model = MoiraiForecast(
            module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.1-R-{SIZE}"),
            prediction_length=PDT,
            context_length=CTX,
            patch_size=PSZ,
            num_samples=5,
            target_dim=1,
            feat_dynamic_real_dim=gluon_data.num_feat_dynamic_real,
            past_feat_dynamic_real_dim=gluon_data.num_past_feat_dynamic_real,
        )
    elif MODEL == "moirai-moe":
        model = MoiraiMoEForecast(
            module=MoiraiMoEModule.from_pretrained(f"Salesforce/moirai-moe-1.0-R-{SIZE}"),
            prediction_length=PDT,
            context_length=CTX,
            patch_size=PSZ,
            num_samples=5,
            target_dim=1,
            feat_dynamic_real_dim=gluon_data.num_feat_dynamic_real,
            past_feat_dynamic_real_dim=gluon_data.num_past_feat_dynamic_real,
        )
    return model

In [117]:
# Moirai-MoE multivariate
model = moirai_maker("moirai-moe", gluon_multi)
predictor = model.create_predictor(batch_size=BSZ)
forecasts = predictor.predict(test_data_multi.input)

input_it = iter(test_data_multi.input)
label_it = iter(test_data_multi.label)
forecast_it = iter(forecasts)

#inp = next(input_it)
#label = next(label_it)
#forecast = next(forecast_it)
results_dict = dict()
letter_sequence = "ABCDEFGHIJ"
for letter in letter_sequence:
    results_dict[letter] = []
    
for fore in forecast_it:
    #print(fore)
    for letter in letter_sequence:
        if fore.item_id == letter:
            #print(fore.samples)
            results_dict[letter].extend(np.median(fore.samples,axis=0))


print(results_dict)

{'A': [0.63682294, -0.08885161, -0.08847554, -0.08917066, 0.35210264, 0.055747792, 0.91144603, 0.05573744], 'B': [0.19806892, 0.19806892, 0.19806892, 0.41025853, 0.67905104, -0.43745625, 0.27192762, -0.43852666], 'C': [0.39108858, 0.7846563, 1.1567093, 0.93468964, 1.3947501, 0.9177928, 0.6662674, 0.665996], 'D': [-0.13739681, 0.60518265, -0.32496133, -0.32720694, -0.20336926, 0.5309346, 2.2014656, -0.20102113], 'E': [-0.17892006, -0.17811641, -0.17801014, -0.17892006, 0.3692882, 0.4324031, -0.3472769, -0.34748352], 'F': [0.70718575, 0.5240841, 0.33650056, 0.33518323, 0.23835024, 0.2381035, 0.07057999, 1.063255], 'G': [0.6813359, 0.49053183, 0.18602249, 0.53956985, 0.5492074, 0.5498458, 0.5482884, -0.1420626], 'H': [0.65762186, 0.46188575, 0.46188575, 0.46118075, 0.91867507, 1.3667681, 0.48583686, 0.48736903], 'I': [0.7494148, 0.34689075, 0.6662913, 0.9471603, -0.017657915, 0.38901538, -0.01728117, -0.01847508], 'J': [0.2069133, 0.061959535, 0.2069133, 1.0570999, 0.5012657, 0.5012657, 0

In [118]:
# Moirai multivariate
model = moirai_maker("moirai", gluon_multi)
predictor = model.create_predictor(batch_size=BSZ)
forecasts = predictor.predict(test_data_multi.input)

input_it = iter(test_data_multi.input)
label_it = iter(test_data_multi.label)
forecast_it = iter(forecasts)

#inp = next(input_it)
#label = next(label_it)
#forecast = next(forecast_it)
results_dict = dict()
letter_sequence = "ABCDEFGHIJ"
for letter in letter_sequence:
    results_dict[letter] = []
    
for fore in forecast_it:
    #print(fore)
    for letter in letter_sequence:
        if fore.item_id == letter:
            #print(fore.samples)
            results_dict[letter].extend(np.median(fore.samples,axis=0))


print(results_dict)

{'A': [0.4199658, -0.04238665, 0.53047466, 0.6484297, -0.7941812, 0.52682906, 0.04704149, -0.35343242], 'B': [-0.47716868, 0.20813932, -0.10218641, 0.38613018, -0.6117233, -0.5441976, -0.6041887, 0.51189494], 'C': [0.47875613, 0.92461705, 0.8862364, 0.5535266, 1.2273306, 2.219431, 1.7001741, 1.509053], 'D': [0.7514059, 1.3698223, -0.12644772, 0.9597598, 0.3858434, -0.91814625, -0.9374661, -1.3722415], 'E': [-0.70506823, -1.3625075, -1.1324836, -2.7330172, -0.3074691, -1.1631563, 0.21918035, 0.8013059], 'F': [0.7243819, 0.46032622, 0.2248562, 0.13675435, -0.03128919, 0.22621758, 0.08883178, 0.5681387], 'G': [0.8100077, 0.54422146, 0.78186285, -0.2545949, 1.5048379, 2.3548255, 0.8634256, 1.9226036], 'H': [0.95227516, 1.6497273, 0.46188575, 1.4687686, 0.81903386, 1.332994, 1.5530438, 0.66443723], 'I': [0.49611318, 0.45553195, 1.3465798, 3.2257514, 0.3974086, 0.12264862, -0.07898504, -0.013508739], 'J': [-0.17340355, 0.12529722, 0.108337365, 0.2834773, -0.014264166, 0.5603231, 0.63579607, 

In [119]:
# Moirai-MoE univariate
model = moirai_maker("moirai-moe", gluon_uni)
predictor = model.create_predictor(batch_size=BSZ)
forecasts = predictor.predict(test_data_uni.input)

input_it = iter(test_data_uni.input)
label_it = iter(test_data_uni.label)
forecast_it = iter(forecasts)

#inp = next(input_it)
#label = next(label_it)
#forecast = next(forecast_it)
results_dict = dict()
letter_sequence = "A"
for letter in letter_sequence:
    results_dict[letter] = []
    
for fore in forecast_it:
    #print(fore)
    for letter in letter_sequence:
        if fore.item_id == letter:
            #print(fore.samples)
            results_dict[letter].extend(np.median(fore.samples,axis=0))


print(results_dict)

{'A': [0.3850466, -0.11534281, -0.1143708, -0.11465937, 0.50890744, -0.0068272552, 0.7272135, 0.0146063175]}


In [120]:
# Moirai univariate
model = moirai_maker("moirai", gluon_uni)
predictor = model.create_predictor(batch_size=BSZ)
forecasts = predictor.predict(test_data_uni.input)

input_it = iter(test_data_uni.input)
label_it = iter(test_data_uni.label)
forecast_it = iter(forecasts)

#inp = next(input_it)
#label = next(label_it)
#forecast = next(forecast_it)
results_dict = dict()
letter_sequence = "A"
for letter in letter_sequence:
    results_dict[letter] = []
    
for fore in forecast_it:
    #print(fore)
    for letter in letter_sequence:
        if fore.item_id == letter:
            #print(fore.samples)
            results_dict[letter].extend(np.median(fore.samples,axis=0))


print(results_dict)

{'A': [-0.21369329, 0.77840966, 0.07158667, -0.74544215, 0.71131486, 0.60488045, 0.5745657, 0.68131053]}
